# Spark  Instantiation

In [1]:
#Ensure we are using the right kernel
spark.version

'3.1.3'

In [2]:
import os
import subprocess
import shutil
import pandas as pd
# import sh
from pyspark.sql.functions import *
#from pyspark.sql import functions as F
from pyspark.sql.types import *

In [3]:
import os
import time
import subprocess

from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [4]:
pd.set_option("max_colwidth", 100)

In [5]:
warnings.filterwarnings(action='ignore')
spark = SparkSession.builder.getOrCreate()

##Add "eagerEval.enabled" to beautify the way Spark DF is displayed
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

## To use legacy casting notation for date
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

# Loading the Data

In [6]:
from google.cloud import storage
# !pip install gcsfs --upgrade

In [7]:
path = "gs://msca-bdp-tweets/final_project/"
bucket = 'msca-bdp-tweets/final_project'

In [8]:
!hadoop fs -ls 'gs://msca-bdp-tweets/final_project/' | head -10

Found 50696 items
-rwx------   3 root root          0 2023-02-08 13:58 gs://msca-bdp-tweets/final_project/_SUCCESS
-rwx------   3 root root    4500466 2023-02-08 13:44 gs://msca-bdp-tweets/final_project/part-00000-aa6d3cb4-7022-4df2-9921-218307589ce2-c000.json
-rwx------   3 root root    4107431 2023-02-08 13:44 gs://msca-bdp-tweets/final_project/part-00001-aa6d3cb4-7022-4df2-9921-218307589ce2-c000.json
-rwx------   3 root root    4672123 2023-02-08 13:44 gs://msca-bdp-tweets/final_project/part-00002-aa6d3cb4-7022-4df2-9921-218307589ce2-c000.json
-rwx------   3 root root    5186684 2023-02-08 13:44 gs://msca-bdp-tweets/final_project/part-00003-aa6d3cb4-7022-4df2-9921-218307589ce2-c000.json
-rwx------   3 root root    4729662 2023-02-08 13:44 gs://msca-bdp-tweets/final_project/part-00004-aa6d3cb4-7022-4df2-9921-218307589ce2-c000.json
-rwx------   3 root root    4605529 2023-02-08 13:44 gs://msca-bdp-tweets/final_project/part-00005-aa6d3cb4-7022-4df2-9921-218307589ce2-c000.json
-rwx-----

In [9]:
%time
tweets_df = spark.read.json(path)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.63 µs


23/02/27 14:30:48 WARN org.apache.spark.sql.execution.datasources.SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.
23/02/27 14:36:47 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [10]:
tweets_df.printSchema()

root
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |    |-- monetizable: boolean (nullable = true)
 |    |    |    |   

In [11]:
#Checking for the total number of records
total_recs = tweets_df.count()
total_recs

99994342

In [12]:
#Checking how many columns are present to proceed with EDA
total_features = len(tweets_df.columns)
total_features

39

In [13]:
tweets_df = tweets_df\
.withColumn('tweet_text', lower('tweet_text'))\
.withColumn('stripped', regexp_replace(col("tweet_text"),"[\$#,&%\".]",""))

# Filtering the text and hashtag columns to get the tweets only related to education (helps in data filtration)

In [31]:
tweets_df = tweets_df.filter(tweets_df.lang == 'en')

In [32]:
tdf_tweet_text = tweets_df.filter(col('tweet_text').rlike('.*tuition.*|.*Tuition.*|.*education.*|.*Education.*|.*school.*|.*School.*|.*college.*|.*College.*|.*university.*|.*University.*|.*grade.*|.*Grade.*'))
tdf_tweet_text.createOrReplaceTempView("tw_df_tweet_text")

In [33]:
tdf_hashtag = spark.sql('select * from tw_df_tweet_text where exists(entities.hashtags.text, x -> x rlike ".*education.*|.*Education.*|.*school.*|.*School.*|.*college.*|.*College.*|.*university.*|.*University.*|.*grade.*|.*Grade.*|.*K12.*|.*k12.*|.*study.*|.*Study.*|.*edchat.*|.*Edchat.*|.*learning.*|.*Learning.*|.*edreform.*|.*Edreform.*|.*scichat.*|.*student.*|.*Student.*")')

In [ ]:
tdf_hashtag.write.format("parquet").\
mode('overwrite').\
save('gs://' + 'msca-bdp-students-bucket/shared_data/kishorkumarreddy' + '/filtered_data_for_analysis_1')

In [ ]:
tdf_hashtag.count()

1005167